<a href="https://colab.research.google.com/github/Duss02/LiveCodeBench-Lite-Evaluation/blob/main/nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets accelerate


In [ ]:
from huggingface_hub import login
import getpass
# Insert your Hugging Face token
token = getpass.getpass("token hf ")
login(token)

In [ ]:
!pip install -q bitsandbytes

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
# Change this to your model link
model_name = "Qwen/Qwen2.5-Coder-3B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map="auto"
)


In [ ]:
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("livecodebench/code_generation_lite", split="test", streaming=True)
print(dataset)

In [ ]:
import torch
import re
import time
from io import StringIO
import contextlib
import traceback
import sys

# --- Configuration ---
MAX_SAMPLES_TO_EVALUATE = 100

def format_prompt(prompt: str) -> str:
    """Formats the prompt for Qwen model"""
    formatted = f"<|im_start|>system\nYou are a helpful coding assistant. Generate only Python code wrapped in ```python ... ``` unless otherwise specified.<|im_end|>\n"
    formatted += f"<|im_start|>user\n{prompt}<|im_end|>\n"
    formatted += f"<|im_start|>assistant\n"
    return formatted

def generate_code(prompt: str, max_new_tokens: int = 400, temperature: float = 0.7, top_p: float = 0.95, timeout_seconds: int = 400) -> str:
    """Generates code using the loaded model with a timeout."""
    global model, tokenizer
    full_prompt = format_prompt(prompt)
    inputs = tokenizer(full_prompt, return_tensors="pt", return_attention_mask=True).to(model.device)
    generate_kwargs = {**inputs, "max_new_tokens": max_new_tokens, "do_sample": True, "temperature": temperature, "top_p": top_p, "pad_token_id": tokenizer.pad_token_id}
    print(f"Generating code for prompt: '{prompt[:50]}...'")
    start_time = time.time()

    try:
        with torch.no_grad():
            output_ids = model.generate(**generate_kwargs, max_time=timeout_seconds)
        end_time = time.time()
        print(f"Generation took {end_time - start_time:.2f} seconds.")
        generated_token_ids = output_ids[0][inputs['input_ids'].shape[1]:]
        generated_text = tokenizer.decode(generated_token_ids, skip_special_tokens=True).strip()
        generated_text = generated_text.replace("<|eot_id|>", "").replace("<|im_end|>", "").strip()
        print(f"Raw generated text:\n---\n{generated_text}\n---")
        return generated_text
    except RuntimeError as e:
        if "Time limit exceeded" in str(e):
            raise TimeoutError(f"Code generation exceeded the time limit of {timeout_seconds} seconds.")
        else:
            raise  # Rilancia altre RuntimeErrors
    except TimeoutError:
        raise

def extract_python_code(generated_text: str) -> str | None:
    """Extracts Python code block, with fallback."""
    code_block_match = re.search(r"```python\n(.*?)\n```", generated_text, re.DOTALL)
    if code_block_match: return code_block_match.group(1).strip()
    else:
        print("Warning: Could not find Python code block ```python ... ```. Assuming entire output is code.")
        if re.search(r"^\s*(def |class |import |from )", generated_text, re.MULTILINE): return generated_text.strip()
        else: print("Warning: Fallback code doesn't look like Python definition/import. Skipping execution."); return None

# --- valuta ---
def evaluate_solution(generated_code: str, test_code: str) -> tuple[bool, str]:
    """
    Executes the generated code and runs the test code against it.
    Prevents blocking hangs by replacing sys.stdin to cause EOFError on input().
    """
    execution_namespace = {}
    stdout_capture = StringIO()
    stderr_capture = StringIO()
    original_stdin = sys.stdin

    try:
        # Questo farà sì che input() sollevi EOFError invece di bloccarsi
        print("Replacing sys.stdin with dummy stream to prevent blocking input() calls.")
        sys.stdin = StringIO('')

        print("Executing generated code...")
        #  redirect_stdout/stderr per catturare l'output
        with contextlib.redirect_stdout(stdout_capture), contextlib.redirect_stderr(stderr_capture):
            exec(generated_code, execution_namespace)

        print("Executing test code...")
        with contextlib.redirect_stdout(stdout_capture), contextlib.redirect_stderr(stderr_capture):
            exec(test_code, execution_namespace)

        print("Tests Passed.")
        return True, f"Passed.\nStdout:\n{stdout_capture.getvalue()}\nStderr:\n{stderr_capture.getvalue()}"

    except EOFError:
        print("Evaluation Failed: Code attempted to read from stdin (e.g., using input()).")
        tb_str = traceback.format_exc()
        return False, f"Failed: Code required user input (EOFError on reading stdin).\nStderr:\n{stderr_capture.getvalue()}\nTraceback:\n{tb_str}"

    except AssertionError as e:
        print(f"Test Failed (AssertionError): {e}")
        tb_str = traceback.format_exc()
        return False, f"Failed: AssertionError: {e}\nStderr:\n{stderr_capture.getvalue()}\nTraceback:\n{tb_str}"
    except SyntaxError as e:
        print(f"Syntax Error in generated code: {e}")
        tb_str = traceback.format_exc()
        return False, f"Failed: SyntaxError: {e}\nStderr:\n{stderr_capture.getvalue()}\nTraceback:\n{tb_str}"
    except Exception as e:
        # Gestisce tutti gli altri errori di esecuzione
        print(f"Execution Error: {e} (Type: {type(e).__name__})")
        tb_str = traceback.format_exc()
        # print(f"Full Traceback:\n{tb_str}")
        return False, f"Failed: Execution Error: {e}\nType: {type(e).__name__}\nStderr:\n{stderr_capture.getvalue()}\nTraceback:\n{tb_str}"
    finally:
        print("Restoring original sys.stdin.")
        sys.stdin = original_stdin

# --- Main ---
results = {}
passed_count = 0
attempted_count = 0

print("\n--- Starting Evaluation from LiveCodeBench Stream ---")
print(f"Will evaluate up to {MAX_SAMPLES_TO_EVALUATE} samples.")

try:
    for i, sample in enumerate(dataset):
        if attempted_count >= MAX_SAMPLES_TO_EVALUATE:
            print(f"\nReached evaluation limit of {MAX_SAMPLES_TO_EVALUATE} samples.")
            break

        attempted_count += 1
        sample_id = sample.get("problem_id", f"streamed_sample_{i}")
        print(f"\n--- Evaluating Sample {attempted_count}/{MAX_SAMPLES_TO_EVALUATE}: {sample_id} ---")

        difficulty = sample.get("difficulty", "Unknown")
        print(f"Problem Difficulty: {difficulty}")

        results[sample_id] = {"passed": False, "message": "Evaluation not started", "generated_code": None, "difficulty": difficulty}

        try:
            problem_prompt = sample["question_content"]
            test_code = sample["public_test_cases"]

            raw_generated_text = generate_code(problem_prompt)
            extracted_code = extract_python_code(raw_generated_text)

            passed = False
            message = "Failed: No valid Python code extracted."
            if extracted_code:
                passed, message = evaluate_solution(extracted_code, test_code)
            else:
                print(message)

            results[sample_id]["passed"] = passed
            results[sample_id]["message"] = message
            results[sample_id]["generated_code"] = extracted_code or raw_generated_text

            if passed:
                passed_count += 1
            print(f"Result for {sample_id}: {'PASSED' if passed else 'FAILED'}")

        except TimeoutError as e:
            error_message = f"Failed: Code generation timed out after 4 minutes."
            print(f"Error: {error_message}")
            results[sample_id]["message"] = error_message
        except KeyError as e:
            error_message = f"Failed: Missing key in dataset sample: {e}. Check dataset structure."
            print(f"Error: Skipping sample {sample_id} due to missing key: {e}. Check dataset structure.")
            results[sample_id]["message"] = error_message
        except Exception as e:
            error_message = f"Failed: Unexpected error during processing: {e}"
            print(f"Error: Skipping sample {sample_id} due to unexpected error during processing: {e}")
            print(traceback.format_exc())
            results[sample_id]["message"] = error_message

except NameError as e:
    print(f"\nError: Variable not defined (likely 'model', 'tokenizer', or 'dataset'): {e}")
    print("Please ensure 'model', 'tokenizer', and 'dataset' are loaded before running the evaluation loop.")
    print(traceback.format_exc())
except Exception as e:
    print(f"\nError loading or iterating through the dataset: {e}")
    print(traceback.format_exc())

print("\n--- Evaluation Complete ---")

if attempted_count > 0:
    pass_at_1 = passed_count / attempted_count
    print(f"\nFinal Pass@1 Score: {pass_at_1:.4f} ({passed_count}/{attempted_count} passed out of {attempted_count} attempted samples)")

    difficulty_results = {}
    for res in results.values():
        diff = res.get("difficulty", "Unknown")
        if diff not in difficulty_results: difficulty_results[diff] = {"passed": 0, "attempted": 0}
        difficulty_results[diff]["attempted"] += 1
        if res["passed"]: difficulty_results[diff]["passed"] += 1

    print("\nPass Rate per Difficulty:")
    for diff, counts in difficulty_results.items():
        if counts["attempted"] > 0:
            pass_rate = counts["passed"] / counts["attempted"]
            print(f"- {diff}: {pass_rate:.4f} ({counts['passed']}/{counts['attempted']})")
        else:
            print(f"- {diff}: No samples attempted")
else:
    print("\nNo samples were attempted evaluation (check dataset loading or iteration).")


import json
with open("result_eval_100.json", "w") as f:
    json.dump(results, f, indent=4)